# Energy Reconstruction Using CNN

In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
import math

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Attention

from tensorflow.keras.callbacks import ModelCheckpoint

from data_tools import load_preprocessed, dataPrep, nameModel

## FIX THIS!
simPrefix = 'D:\icecube\sim_data'

In [2]:
# Name for model
key = 'CNN_attn'

# Data preparation: no merging of charge (q), no time layers included (t=False), data normalized from 0-1
prep = {'q':None, 't':False, 'normed':True} #i dont understand exactly what all of these options are

In [3]:
x, y = load_preprocessed(simPrefix, 'train')
print(y.keys())

Percentage of events with a NaN: 2.68
dict_keys(['comp', 'energy', 'dir', 'plane_dir', 'laputop_dir', 'small_dir'])


In [4]:
i = 0
print(len(y['dir']))
print(type(y['dir']))



all_directions = y['dir']
cosed_all_directions = []

for direction_arrays in all_directions:
    cosed_all_directions.append([math.cos(direction_arrays[0]), direction_arrays[1]])#run cosine on zenith (i think)
    

    
y['dir']= np.array(cosed_all_directions)
       


549773
<class 'numpy.ndarray'>


In [5]:
model = Sequential(name = nameModel(prep, 'test')) # i dont know what the nameModel object does exactly
model.add(Conv2D(64, kernel_size=3, activation='relu', input_shape=(10,10,2)))
model.add(MaxPooling2D(pool_size = (2,2), strides = None))
model.add(Conv2D(64, kernel_size=3, activation='relu', input_shape=(10,10,2)))
model.add(Flatten())
model.add(Attention(use_scale = False))
model.add(Dense(1))

model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse'])




ValueError: Attention layer must be called on a list of inputs, namely [query, value] or [query, value, key].

In [ ]:
energy = y['energy']
comp = y['comp']
theta, phi = y['dir'].transpose()
nevents = len(energy)
trainCut = (np.random.uniform(size=nevents) < 0.85)
testCut = np.logical_not(trainCut)

# Establish arrays to be trained on
x_i = dataPrep(x, y, **prep)
temp_y = energy


In [ ]:
history = model.fit(x_i[trainCut], temp_y[trainCut], validation_data=(x_i[testCut], temp_y[testCut]), epochs=3)



In [ ]:
# Save model to file for easy loading
## WHERE ARE YOU SAVING TO?
model.save('model_%s.h5' % key)